In [21]:
# Parameters
country_name = "Zimbabwe"


# Optimization Problem Time Series

This notebook optimizes an electrci mix based on : 
- an electricity demand time serie
- a PV production time serie
- a wind production time serie

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pulp import *
import sys
from utilities import import_excel, optimize_enr, format_load_data
import plotly.graph_objects as go
import plotly.express as px
import pickle
from ren_ninja_api import fetch_and_average_data_ren_ninja, get_regular_coordinates
import plotly.io as pio
import os
import glob
import geopandas as gpd
pio.renderers.default='notebook'

Enter country name for file naming: 

In [25]:
year = 2021
# country_name = 'Russia'
country_name = country_name
# Directory path
path_input_data = '../input_time_series/'
mode = 'grid'

In [26]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")
transformer.transform(12, 12)

(1335833.8895192828, 1345708.4084091089)

In [27]:
points_in_world = gpd.read_file('grid_with_centroids.geojson')

### Load Time Series

First we load the time series that will be used in the problem. We use the  ```import_excel``` function used in the wavelet decomposition.

#### Demand

In [28]:
# Demand time serie

dpd = 24 # data per day in the time serie
dpy = 365 # data per year :  cut the leap years to 365 years

ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

file_name = format_load_data(country_name)
Load_ts = import_excel(path_input_data,file_name, 
                                    dpd ,ndpd, dpy, 
                                    interp=True, norm = 'mean') # interpolate data from dpd to ndpd numper of points per day

mean_load = pd.read_excel(path_input_data+file_name).mean().iloc[0]

#### Wind production

In [29]:
# Wind time serie 

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years

# We interpolate so that we have hourly data
ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

folder = path_input_data + f'/{country_name}'
partie_name_file = f'{mode}_locations_averaged_wind_{country_name}_{year}.xlsx'
chemin_pattern = os.path.join(folder, f'*{partie_name_file}*')
fichiers_trouves = glob.glob(chemin_pattern)
print(fichiers_trouves)

if len(fichiers_trouves)==0:
    print('collecting data')
    fetch_and_average_data_ren_ninja(country_name, 1, ['wind'], points_in_world, year=year, save = True, coordinates = mode)

fichiers_trouves = glob.glob(chemin_pattern)
file_name = fichiers_trouves[0].split('/',2)[-1]
Wind_ts = import_excel(path_input_data,file_name, 
                                dpd ,ndpd, dpy, 
                                interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_wind = pd.read_excel(path_input_data+file_name).mean().iloc[0]

[]
collecting data
Error fetching data for location (60.0, 30.0) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2379 seconds.
Error fetching data for location (50.0, 40.0) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2378 seconds.
Error fetching data for location (60.0, 40.0) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2377 seconds.
Error fetching data for location (60.0, 50.0) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2375 seconds.
Error fetching data for location (60.0, 60.0) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2374 seconds.
Error fetching data for location (60.0, 70.0) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2373 seconds.
Error fetching data for location (70.0, 70.0) with technology wind: Error 429: Error: Reached limit (50/h

IndexError: list index out of range

#### PV production

In [18]:
# PV time serie

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years


# We interpolate so that we have hourly data
ndpd = 24 # new data per day (for the interpolation)
signal_length = ndpd * dpy

folder = path_input_data + f'/{country_name}'
partie_name_file = f'{mode}_locations_averaged_pv_{country_name}_{year}.xlsx'
chemin_pattern = os.path.join(folder, f'*{partie_name_file}*')
fichiers_trouves = glob.glob(chemin_pattern)
print(fichiers_trouves)

if len(fichiers_trouves)==0:
    print('collecting data')
    fetch_and_average_data_ren_ninja(country_name, 1, ['pv'], points_in_world, year=year, save = True, coordinates = mode)

fichiers_trouves = glob.glob(chemin_pattern)
file_name = fichiers_trouves[0].split('/',2)[-1]
PV_ts = import_excel(path_input_data,file_name, 
                                dpd ,ndpd, dpy, 
                                interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_pv = pd.read_excel(path_input_data+file_name).mean().iloc[0]

collecting data


### Plot Time Series

In [19]:
# # Create a Plotly figure
# fig = go.Figure()

# fig.add_trace(go.Scatter(y=PV_ts, mode='lines', name='PV',marker=dict(color=colors_dict['PV'])))
# fig.add_trace(go.Scatter(y=Load_ts, mode='lines', name='Demand',marker=dict(color=colors_dict['Consumption'])))
# fig.add_trace(go.Scatter(y=Wind_ts, mode='lines', name='Wind',marker=dict(color=colors_dict['Wind'])))
# fig.update_layout(title=f'{country_name} 2021', xaxis_title='Day', yaxis_title='Energy normalized')

# # Show the plot
# fig.show()

### Description of the problem

#### Equations:

- **Objective function** : 
  - Minimize dispatchable energy: $ \min(\sum{P_{dispatchable}(t)*dt}) $


- **Node Law** : 
  - $(P_{pv}(t) + P_{wind}(t) + P_{dispatchable}(t) - P_{in\_stock}(t) + P_{out\_stock}(t) = P_{demand}(t) + P_{curt}(t))$


- **State of charge**
  - $SOC(t+1)=SOC(t)+P_{in\_stock}(t) - P_{out\_stock}(t)$

#### Contraintes :
- $E_{wind} + E_{pv} \leq E_{demand}$
- We want a maximum storage size of 10 hours:  $E_{stock} \leq 10$
- Charging and discharging at the same time is not possible. 


### Implementation
#### Decision Variables:
- `x_pv`: Installed capacity for photovoltaic production.
- `x_wind`: Installed capacity for wind production.
- `ts_dispatchable`: Dispatchable production (can be controlled), time serie.
- `p_ch`: Battery charging power, time serie.
- `p_dech`: Battery discharging power, time serie.
- `SOC_ts`: State of charge of the battery, time serie.
- `p_curt`: Curtailment power (lost energy), time serie.
- `dech_active`: Binary variable indicating if the battery is charging or discharging.

### Run the optimization with GUROBI

**If the optimization has already been run, go to the next part where results can be loaded and analysed in plots.**

In [20]:
optimized_parameters=optimize_enr(country_name, Load_ts, PV_ts, Wind_ts, mean_load, save_results = True)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2503220 - for non-commercial use only - registered to du___@ethz.ch
Optimize a model with 35041 rows, 52561 columns and 120314 nonzeros
Model fingerprint: 0x91297196
Variable types: 43802 continuous, 8759 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+05]
  RHS range        [7e-01, 1e+05]
Found heuristic solution: objective 8760.0000000
Presolve removed 5 rows and 8765 columns
Presolve time: 0.66s
Presolved: 35036 rows, 43796 columns, 106348 nonzeros
Found heuristic solution: objective 8759.1434796
Variable types: 35037 continuous, 8759 integer (8759 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Sho

### Add results to .csv file with all countries

In [21]:
df_new = pd.DataFrame({"Country" : [country_name],"iso_alpha":[optimized_parameters['iso_alpha']],
    "mean_load":[optimized_parameters['mean_consumption']],
    "E_dispatch": [optimized_parameters['E_dispatch']],
    "P_dispatch": [optimized_parameters['dispatchable_capacity']],
    "E_destock": [optimized_parameters['E_destock']],
    "P_pv": [optimized_parameters['pv_capacity']],
    "P_wind": [optimized_parameters['wind_capacity']]
})


In [22]:
all_results_file = "results/optimization_results_world_grid.csv"
file_exists = os.path.isfile(all_results_file)

df_new.to_csv(all_results_file, mode='a', index=False, header=not file_exists)

print(f"Data have been added to {all_results_file}")    

Data have been added to results/optimization_results_world_grid.csv


## Plots

In [13]:
# from plots import plot_ts_optim, plot_pie_energy, plot_storage, plot_stack_production

In [14]:
# plot_ts_optim([optimized_parameters['optimized_pv'], optimized_parameters['optimized_wind'], optimized_parameters['optimized_dispatchable'], optimized_parameters['optimized_p_curt'],np.array(Load_ts) ], ['PV', 'Wind', 'Dispatchable', 'Curtailment', 'Consumption'], country_name, colors_dict = colors_dict,savefig=False)

In [15]:
# plot_storage(optimized_parameters['optimized_charge'], optimized_parameters['optimized_discharge'], optimized_parameters['optimized_stock'], country_name, colors_dict = colors_dict, savefig=False)

In [16]:
# E_wind = optimized_parameters['E_wind']
# E_pv = optimized_parameters['E_pv']
# E_dispatch = optimized_parameters['E_dispatch']

In [17]:
# plot_pie_energy([E_wind, E_pv, E_dispatch], country_name, colors_dict =colors_dict, savefig=False)